# Define molecule

In [ ]:
# ==> Import statements & Global Options <==
import psi4
import numpy as np

# Set Psi4 memeory as about 2GB
psi4.set_memory(int(2e9))
numpy_memory = 2

# Set output file as output.dat
psi4.core.set_output_file('output.dat', False)

In [ ]:
# ==> Molecule & Psi4 Options Definitions <==
mol = psi4.geometry("""
    O  0.000000000000    -0.000000000000    -0.079135765807
    H  0.000000000000     0.707106781187     0.627971015380
    H  0.000000000000    -0.707106781187     0.627971015380
    symmetry c1
""")

psi4.set_options({'basis':        '6-31g',
                  'scf_type':     'pk',
                  'mp2_type':     'conv',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8,
                  'dft_spherical_points': 590,
                  'dft_radial_points': 99})

In [ ]:
# Get the SCF wavefunction & energies
scf_e, scf_wfn = psi4.energy('B3LYP', return_wfn=True)

In [ ]:
# B3LYP Energy
scf_e

# MP2 part of XYG3

In [ ]:
# ==> ERIs <==
# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(scf_wfn.basisset())

In [ ]:
# ==> Get orbital information & energy eigenvalues <==
# Number of Occupied orbitals & MOs
ndocc = scf_wfn.nalpha()
nmo = scf_wfn.nmo()

# Get orbital energies, cast into NumPy array, and separate occupied & virtual
eps = np.asarray(scf_wfn.epsilon_a())
e_ij = eps[:ndocc]
e_ab = eps[ndocc:]

In [ ]:
# ==> Construct MO integrals (ia|jb) = <ij|ab> <==
Co = scf_wfn.Ca_subset('AO','OCC')
Cv = scf_wfn.Ca_subset('AO','VIR')
MO = np.asarray(mints.mo_eri(Co, Cv, Co, Cv))

In [ ]:
# ==> Compute MP2 Correlation & MP2 Energy <==
# Compute energy denominator array
e_denom = 1 / (e_ij.reshape(-1, 1, 1, 1) - e_ab.reshape(-1, 1, 1) + e_ij.reshape(-1, 1) - e_ab)

# Compute SS & OS MP2 Correlation with Einsum
mp2_os_corr = np.einsum('iajb,iajb,iajb->', MO, MO, e_denom)
mp2_ss_corr = np.einsum('iajb,iajb,iajb->', MO, MO - MO.swapaxes(1,3), e_denom)

# Total MP2 Energy
MP2_corr = mp2_os_corr + mp2_ss_corr

In [ ]:
# ==> Compute Scaled MP2 Correlation <==
MP2_corr * 0.3211

# B3LYP Energy Decomposition

### XC Part

XC part of B3LYP energy can be calculated when forming XC potential $ V_{\mu \nu}^\mathrm{xc} [ \mathbf{P}^\mathsf{AO} ] $

In [ ]:
# Number of basis functions
nbf = scf_wfn.nmo()
# Empty matrix; XC potential (contribution to Fock matrix) should be stored in this variable
#   once `compute_V' is called
V = psi4.core.Matrix(nbf, nbf)
# DFT potential calculation engine
Vpot = scf_wfn.V_potential()
# SCF AO density
D = scf_wfn.Da()

# Set AO density to DFT potential engine
Vpot.set_D([D])
# Calculate XC potential, meanwhile energy contribution of XC part to total B3LYP energy
#   is also obtained
Vpot.compute_V([V])
# XC energy can be dumped by the following method
Vpot.quadrature_values()["FUNCTIONAL"]

Comparing to the energy calculated by Psi4

In [ ]:
scf_wfn.get_energies("XC")

### One Electron Part

Energy of core Hamiltonian can be obtained by $ E_{\hat H^\mathrm{core}} = H_{\mu \nu}^\mathrm{core} P_{\mu \nu} $

In [ ]:
# Obtain core Hamiltonian in AO basis
H = np.asarray(scf_wfn.H())

In [ ]:
# Calculate 
np.einsum("ij, ij ->", H, D) * 2

### Two Electron Part

Coulomb and exchange contribution to energy can be obtained by

$ J = P_{\mu \nu} (\mu \nu | \kappa \tau) P_{\kappa \tau} $

$ K = P_{\mu \kappa} (\mu \nu | \kappa \tau) P_{\nu \tau} $

Since $ c_\mathrm{x} = 0.2 $ when using B3LYP, so two electron contribution to energy is $ 2 J - c_\mathrm{x} K $ when RHF reference.

J Part

In [ ]:
# Atomic integral in (pq|rs)
I = np.asarray(mints.ao_eri())
# two electron part: 2 J - 0.2 K
2 * np.einsum("pqrs, pq, rs ->", I, D, D, optimize=True) \
- np.einsum("pqrs, pr, qs ->", I, D, D, optimize=True) * 0.2

### Total Energy

All energy contributions have been calculated. We can compare the following result to the B3LYP energy.

In [ ]:
scf_wfn.get_energies("Nuclear") \
+ np.einsum("ij, ij ->", H, D) * 2 \
+ 2 * np.einsum("pqrs, pq, rs ->", I, D, D, optimize=True) \
- np.einsum("pqrs, pr, qs ->", I, D, D, optimize=True) * 0.2 \
+ Vpot.quadrature_values()["FUNCTIONAL"]

# XYG3 Non-Consistent Part

First, we need to define XYG3 non-consistent DFT energy form:

In [ ]:
# ==> Define Functional <===
def build_xyg3_nc_superfunctional(name, npoints, deriv, restricted):

    # Build a empty superfunctional
    sup = psi4.core.SuperFunctional.blank()

    # No spaces, keep it short and according to convention
    sup.set_name('XYG3NC')
    sup.set_description('    XYG3 Non-Consistent Functional without MP2 Part\n')
    
    # -0.0140 * LDA Exchange
    # -0.0140 = 1 - 0.8033 - 0.2107
    lda_x = psi4.core.LibXCFunctional("XC_LDA_X", restricted)
    lda_x.set_alpha(-0.0140)
    sup.add_x_functional(lda_x)
    
    # 0.2107 * B88 Exchange
    gga_x = psi4.core.LibXCFunctional("XC_GGA_X_B88", restricted)
    gga_x.set_alpha(0.2107)
    sup.add_x_functional(gga_x)
    
    # 0.6789 * LYP Correlation
    # 0.6789 = 1 - 0.3211
    lyp_c = psi4.core.LibXCFunctional("XC_GGA_C_LYP", restricted)
    lyp_c.set_alpha(0.6789)
    sup.add_c_functional(lyp_c)
    
    # 0.8033 Exact Exchange
    sup.set_x_alpha(0.8033)

    return sup

Although we do not need to make calculation using XYG3 non-consistent functional. However, since wavefunction can be obtained after an SCF job, I just do one energy calculation for convenience.

This energy calculation is only to obtain wavefunction and thus DFT energy and potential calculation engine.

In [ ]:
nscf_e, nscf_wfn = psi4.energy("SCF", dft_functional=build_xyg3_nc_superfunctional, return_wfn=True)

In [ ]:
# Although we can get XC part of energy
# !!! THIS VALUE CAN NOT BE USED !!!
# since we are doing SCF calculation where XC defined as XYG3 non-consistent
nscf_wfn.get_energies("XC")

In [ ]:
# Non-consistent potential
Vn = psi4.core.Matrix(nbf, nbf)
# DFT potential calculation engine of XYG3 XC form
Vnpot = nscf_wfn.V_potential()

# Set **SCF** AO density to XYG3 **non-SCF** potential engine
Vnpot.set_D([D])
Vnpot.compute_V([Vn])
# The following value is non-consistent XC contribution to XYG3 energy
Vnpot.quadrature_values()["FUNCTIONAL"]

Then Total energy can be obtained by the following code, using SCF AO density:

In [ ]:
xyg3_e = scf_wfn.get_energies("Nuclear")
xyg3_e += np.einsum("ij, ij ->", H, D) * 2
xyg3_e += 2 * np.einsum("pqrs, pq, rs ->", I, D, D, optimize=True)
xyg3_e -= np.einsum("pqrs, pr, qs ->", I, D, D, optimize=True) * 0.8033
xyg3_e += Vnpot.quadrature_values()["FUNCTIONAL"]
xyg3_e += MP2_corr * 0.3211

In [ ]:
# Compare that to Gaussian within 6 decimals
psi4.compare_values(xyg3_e, -0.76282393305943E+02, 6, 'XYG3 Energy')